In [13]:
import numpy as np
import xarray as xr
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

import ThetaoSectors as TS

In [14]:
run_dir = '/scratch/nk0j/ecearth3-cmip6'
leg_number= '001'
exp_name = '5icc'
year = 1850
file_thetao = f'{run_dir}/{exp_name}/output/nemo/{leg_number}/{exp_name}_1m_{year}0101_{year}1231_opa_grid_T_3D.nc' #other output format
print(file_thetao)

/scratch/nk0j/ecearth3-cmip6/5icc/output/nemo/001/5icc_1m_18500101_18501231_opa_grid_T_3D.nc


In [15]:
ds = xr.open_dataset(file_thetao)
ds

<xarray.Dataset>
Dimensions:               (y: 292, x: 362, olevel: 75, axis_nbounds: 2,
                           time_counter: 12)
Coordinates:
    nav_lat               (y, x) float32 ...
    nav_lon               (y, x) float32 ...
  * olevel                (olevel) float32 0.5058 1.556 ... 5.698e+03 5.902e+03
    time_centered         (time_counter) datetime64[ns] ...
  * time_counter          (time_counter) datetime64[ns] 1850-01-16T12:00:00 ....
Dimensions without coordinates: y, x, axis_nbounds
Data variables:
    olevel_bounds         (olevel, axis_nbounds) float32 ...
    time_centered_bounds  (time_counter, axis_nbounds) datetime64[ns] ...
    time_counter_bounds   (time_counter, axis_nbounds) datetime64[ns] ...
    agessc                (time_counter, olevel, y, x) float32 ...
    masscello             (time_counter, olevel, y, x) float32 ...
    obvfsq                (time_counter, olevel, y, x) float32 ...
    so                    (time_counter, olevel, y, x) float32 ...
    thetao                (time_counter, olevel, y, x) float32 ...
    thkcello              (time_counter, olevel, y, x) float32 ...
    zhalfo                (time_counter, olevel, y, x) float32 ...
Attributes:
    name:         5icc_1m_18500101_18501231_opa_grid_T_3D
    description:  Created by xios
    title:        Created by xios
    Conventions:  CF-1.6
    timeStamp:    2023-Jun-05 14:37:49 GMT
    uuid:         63697872-dfa4-47e8-8323-f55a827f23af

In [16]:
ds = ds.rename({'y':'j','x':'i','nav_lon':'longitude','nav_lat':'latitude','olevel':'lev'})

## Compute time mean value over annual file 
ds_thetao_year = ds['thetao'].mean('time_counter') #needs to be more accurate (taking into account days per month)

# Read lev bnds
ds_lev_bnds = ds['olevel_bounds']

In [23]:
ds_lev_bnds[:,1].values

array([1.0239066e+00, 2.1031899e+00, 3.2513092e+00, 4.4850526e+00,
       5.8252378e+00, 7.2974429e+00, 8.9326859e+00, 1.0767898e+01,
       1.2845992e+01, 1.5215269e+01, 1.7927923e+01, 2.1037571e+01,
       2.4595987e+01, 2.8649651e+01, 3.3236965e+01, 3.8387100e+01,
       4.4121010e+01, 5.0454468e+01, 5.7402565e+01, 6.4984604e+01,
       7.3228699e+01, 8.2175560e+01, 9.1881409e+01, 1.0242017e+02,
       1.1388524e+02, 1.2639093e+02, 1.4007397e+02, 1.5509505e+02,
       1.7164024e+02, 1.8992279e+02, 2.1018446e+02, 2.3269698e+02,
       2.5776288e+02, 2.8571579e+02, 3.1691986e+02, 3.5176801e+02,
       3.9067862e+02, 4.3409055e+02, 4.8245633e+02, 5.3623322e+02,
       5.9587207e+02, 6.6180524e+02, 7.3443213e+02, 8.1410565e+02,
       9.0111798e+02, 9.9568854e+02, 1.0979541e+03, 1.2079630e+03,
       1.3256725e+03, 1.4509504e+03, 1.5835824e+03, 1.7232798e+03,
       1.8696931e+03, 2.0224249e+03, 2.1810444e+03, 2.3451011e+03,
       2.5141372e+03, 2.6876995e+03, 2.8653469e+03, 3.0466592e

In [26]:
bm_dep1 = 200
bm_dep2 = 700

In [30]:
d = 0
for depth in [bm_dep1, bm_dep2]:
    # Find oceanic layers that sit fully within the depth bounds
    if d == 0:
        lev_ind = TS.nearest_above(ds_lev_bnds[:,0],depth) #above = deeper than
        depth_nemo = ds_lev_bnds[lev_ind,0].values
        print('Upper bound of ocean layer used for basal melt distribution: ' , depth_nemo, ' m')
    elif d == 1:
        lev_ind= TS.nearest_below(ds_lev_bnds[:,1],depth) #below = shallower than
        depth_nemo = ds_lev_bnds[lev_ind,1].values
        print('Lower bound of ocean layer used for basal melt distribution: ' , depth_nemo, ' m')
    d += 1

Upper bound of ocean layer used for basal melt distribution:  210.18446  m
Lower bound of ocean layer used for basal melt distribution:  661.80524  m
661.80524
